## Red Neuronal Básica para Clasificación con Keras

## Importar Librerias

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.pipeline import Pipeline

## Generar un dataset sintético
Datos artificiales que simulan problemas de clasificación.

- n_samples=200: Genera 200 ejemplos o filas (puntos de datos).
- n_features=2:Cada ejemplo tiene 2 características (columnas), es decir, X.shape = (200, 2).
- n_informative=2: Las 2 características realmente contienen información útil para separar las clases.
- n_redundant=0: No se incluyen características redundantes (combinaciones lineales de las informativas).
- n_clusters_per_class=1: Cada clase se genera a partir de 1 grupo o clúster (simplifica la visualización).
- random_state=42: Semilla para que el resultado sea reproducible (siempre se generan los mismos datos).


In [ ]:
# Generar 500 muestras de 2 caracteristicas, ambas información útil, no redundante, con 1 cluster por clase. 
X, y = make_classification(n_samples=500, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=42)

## Visualización de los datos

In [ ]:
# Generar grafica de dispersión 
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title("Datos sintéticos para clasificación")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

# Dividir en entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Establecer estandarización y clasificación

In [ ]:
# Escalar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Construir el modelo
model = Sequential()
model.add(Dense(10, input_dim=2, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

## Entrenar el modelo

In [ ]:
model.compile(optimizer=Adam(0.01), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=0)

## Evaluar

In [ ]:
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype("int32")
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap="Blues")
plt.title("Matriz de confusión")
plt.grid(False)
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Precisión en test: {accuracy:.2f}")

## Visualización de la frontera de decisión

In [ ]:
h = .02
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
X_grid = np.c_[xx.ravel(), yy.ravel()]
X_grid_scaled = scaler.transform(X_grid)
Z = (model.predict(X_grid_scaled) > 0.5).astype("int32")
Z = Z.reshape(xx.shape)

plt.figure(figsize=(8,6))
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.4)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, edgecolor='k')
plt.title("Frontera de decisión - Red Neuronal")
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()

## Ejercicios
- Cambiar el número de capas o neuronas.
- Probar con diferentes funciones de activación (tanh, sigmoid).
- [dataset para predecir la especie de algunos pingüinos](https://www.openml.org/d/42585) (Aplicar este modelo al dataset penguins)

In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd

dataset = fetch_openml('penguins', as_frame=False)

data = pd.DataFrame(
    dataset.data,
    columns=dataset.feature_names
)

data['species'] = dataset.target

data.head()